In [2]:
import datetime as dt
import utils
import pandas as pd
import icharts
from functools import cache


TEST_START = dt.datetime.strptime("2023-01-01", "%Y-%m-%d")
TEST_END = dt.datetime.strptime("2023-12-31", "%Y-%m-%d")
SYMBOL = "NIFTY 50"
IC_SYMBOL = "NIFTY"
INTERVAL = utils.INTERVAL_MIN1
EXCHANGE = utils.EXCHANGE_NSE
pickle_file_name = "test_ocdf_2024_02_17.pkl"

pd.set_option("display.max_colwidth", None)
pd.set_option("display.max_rows", 200)


def build_date_range(date_start, date_end, symbol):
    date_range = []
    cur_date = date_start
    while cur_date < date_end:
        if cur_date.weekday() not in [5, 6]:
            has_data, _ = utils.has_data(symbol, cur_date, interval=INTERVAL, exchange=EXCHANGE)
            if has_data:
                date_range.append(cur_date)
        cur_date += dt.timedelta(days=1)
    return date_range

# all_dates = pd.DataFrame({"trade_date": build_date_range(TEST_START, TEST_END, SYMBOL)})
# all_dates_shuffled = all_dates.sample(frac=1, random_state=42)

# train_size = int(0.5 * len(all_dates_shuffled))
# # train_dates = all_dates_shuffled.iloc[:train_size]
# test_dates = all_dates_shuffled.iloc[train_size:]
# # train_dates = train_dates.sort_values(by="trade_date")
# # train_dates.set_index("trade_date", inplace=True)
# test_dates = test_dates.sort_values(by="trade_date")
# test_dates.set_index("trade_date", inplace=True)

In [2]:
@cache
def get_intraday_data(date):
    return utils.get_data(symbol=SYMBOL, date=date, interval=INTERVAL, exchange=EXCHANGE)

@cache
def get_symbol_first_candle_open(symbol, trade_date):
    data = utils.get_data(symbol=SYMBOL, date=trade_date, interval=INTERVAL, exchange=EXCHANGE)
    return data.iloc[0].open

@cache
def get_last_trading_day(date):
    return utils.get_last_trading_day(SYMBOL, date, interval=INTERVAL, exchange=utils.EXCHANGE_NSE)

test_dates["previous_trading_day"] = None
test_dates["previous_trading_day"] = test_dates.apply(lambda row: get_last_trading_day(row.name), axis=1)
test_dates["expiry"] = test_dates.apply(lambda row: utils.find_closest_expiry(SYMBOL, row.name), axis=1)
test_dates["market_open"] = test_dates.apply(lambda row: get_symbol_first_candle_open(IC_SYMBOL, row.name), axis=1)

result = []
for i, row in test_dates.iterrows():
    ocdf = icharts.get_oc_df(IC_SYMBOL, row.expiry, row.previous_trading_day)
    ocdf = ocdf.loc[((row.market_open - 700) < ocdf.index) & ((row.market_open + 700) > ocdf.index)]
    ocdf.loc[:,"trade_date"] = row.name
    for col, val in row.items():
        ocdf[col] = val
    result.append(ocdf)
ocdf = pd.concat(result)

ocdf["previous_trading_candles"] = ocdf.apply(lambda row: get_intraday_data(row.previous_trading_day), axis=1)
ocdf["previous_trading_open"] = ocdf.apply(lambda row: row["previous_trading_candles"].iloc[0].open, axis=1)
ocdf["previous_trading_close"] = ocdf.apply(lambda row: row["previous_trading_candles"].iloc[-1].close, axis=1)
del ocdf["previous_trading_candles"]
# ocdf["previous_day_option_chain_file_path"] = ocdf.apply(lambda row: icharts.get_option_chain_file_path(symbol=SYMBOL, expiry=row.expiry, cur_dt=row.previous_trading_day), axis=1)
# test_dates["pdoc"] = test_dates.apply(lambda row: icharts.get_oc_df(IC_SYMBOL, row.expiry, row.previous_trading_day), axis=1) # pdoc - Previous day option chain
ocdf["first_candle_open"] = ocdf["market_open"]
ocdf["market_open_pt"] = ocdf["market_open"] - ocdf["previous_trading_close"]
ocdf["market_open_pc"] = ocdf["market_open_pt"] / ocdf["previous_trading_open"]

get_intraday_data.cache_clear()
get_symbol_first_candle_open.cache_clear()
get_last_trading_day.cache_clear()

In [3]:

def calculate_expected_premium(r, delta, theta, market_open_pt):
    return delta * market_open_pt + theta

ocdf["ec_ce_pt"] = ocdf.apply(lambda r: calculate_expected_premium(r, r.ce_delta, r.ce_theta, r.market_open_pt), axis=1) # ec - expected points change in premium
ocdf["ec_pe_pt"] = ocdf.apply(lambda r: calculate_expected_premium(r, r.pe_delta, r.pe_theta, r.market_open_pt), axis=1) # ec - expected points change in premium
ocdf["ec_ce_pc"] = ocdf["ec_ce_pt"] / ocdf["ce_ltp"]
ocdf["ec_pe_pc"] = ocdf["ec_pe_pt"] / ocdf["pe_ltp"]
ocdf

,ce_build_up,ce_trend,ce_time,ce_vega,ce_theta,ce_gamma,ce_delta,ce_iv_chg_pc,ce_iv_chg,ce_iv,...,market_open,previous_trading_open,previous_trading_close,first_candle_open,market_open_pt,market_open_pc,ec_ce_pt,ec_pe_pt,ec_ce_pc,ec_pe_pc
strike_price,,,,,,,,,,,,,,,,,,,,,
17500,LB,Bullish,02-Jan-2023_EOD,0.0370,-0.0852,0.0000,0.9994,-,-,18.42,...,18163.20,18131.7,18207.1,18163.20,-43.90,-0.002421,-43.958860,-1.634250,-0.061224,-0.797195
17550,LB,Bullish,02-Jan-2023_EOD,0.0752,-0.1729,0.0000,0.9986,-,-,20.54,...,18163.20,18131.7,18207.1,18163.20,-43.90,-0.002421,-44.011440,-1.917520,-0.065689,-0.913105
17600,LB,Bullish,02-Jan-2023_EOD,0.1448,-0.3328,0.0000,0.9971,58.67,7.04,19.04,...,18163.20,18131.7,18207.1,18163.20,-43.90,-0.002421,-44.105490,-1.980430,-0.071155,-0.707296
17650,LB,Bullish,02-Jan-2023_EOD,0.2642,-0.6072,0.0001,0.9944,60.21,7.52,20.01,...,18163.20,18131.7,18207.1,18163.20,-43.90,-0.002421,-44.261360,-1.876330,-0.077306,-0.625443
17700,SC,Bullish,02-Jan-2023_EOD,0.4572,-1.0507,0.0001,0.9896,23.58,3.00,15.72,...,18163.20,18131.7,18207.1,18163.20,-43.90,-0.002421,-44.494140,-2.229330,-0.085689,-0.543739
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22200,LB,Bullish,28-Dec-2023_EOD,7.7269,-7.1584,0.0007,0.1728,0.23,0.03,12.97,...,21737.65,21715.0,21773.7,21737.65,-36.05,-0.001660,-13.387840,22.046625,-0.371369,0.060402
22250,LB,Bullish,28-Dec-2023_EOD,6.9619,-6.5740,0.0006,0.1473,4.42,0.56,13.22,...,21737.65,21715.0,21773.7,21737.65,-36.05,-0.001660,-11.884165,24.048470,-0.394823,0.052634
22300,LB,Bullish,28-Dec-2023_EOD,6.0894,-5.7588,0.0005,0.1212,4.50,0.57,13.24,...,21737.65,21715.0,21773.7,21737.65,-36.05,-0.001660,-10.128060,25.606370,-0.426445,0.051090


In [4]:
def ct(fn):
    def wraps(*args, **kwargs):
        t1 = dt.datetime.now()
        fn(*args, **kwargs)
        t2 = dt.datetime.now()
        print(round((t2-t1).total_seconds(), 2))
    return wraps

print(ocdf.shape)
def get_market_open_ohlc(expiry, trade_date, strike_price, option_type):
    def get_premium_df():
        try:
            pr = icharts.get_opt_pre_df(SYMBOL, expiry, trade_date, strike_price, option_type)
        except FileNotFoundError:
            # print(f"ex: {expiry}, sp:{strike_price}, td:{trade_date}")
            return pd.NA
        return pr
    opdf = get_premium_df()
    if type(opdf) == type(pd.NA):
        return pd.NA
    date = trade_date.replace(hour=9, minute=15, second=0)
    last_candles = opdf[opdf.index >= date]
    if last_candles.shape[0] > 0:
        last = last_candles.iloc[0]
        return last.open
    return pd.NA

# ocdf["ce_first_candle"] = ocdf.apply(lambda r: get_market_open_ohlc(r.expiry, r.trade_date, r.name, icharts.OPTION_TYPE_CALL), axis=1)
# ocdf["pe_first_candle"] = ocdf.apply(lambda r: get_market_open_ohlc(r.expiry, r.trade_date, r.name, icharts.OPTION_TYPE_PUT), axis=1)

(3444, 69)


In [6]:
ocdf["ce_first_candle_open"] = ocdf.apply(lambda r: get_market_open_ohlc(r.expiry, r.trade_date, r.name, icharts.OPTION_TYPE_CALL), axis=1)
ocdf["pe_first_candle_open"] = ocdf.apply(lambda r: get_market_open_ohlc(r.expiry, r.trade_date, r.name, icharts.OPTION_TYPE_PUT), axis=1)

ocdf["ce_actual_chg_pt"] = ocdf["ce_first_candle_open"] - ocdf["ce_ltp"]
ocdf["ce_actual_chg_pc"] = ocdf["ce_actual_chg_pt"] / ocdf["ce_ltp"]

ocdf["pe_actual_chg_pt"] = ocdf["pe_first_candle_open"] - ocdf["pe_ltp"]
ocdf["pe_actual_chg_pc"] = ocdf["pe_actual_chg_pt"] / ocdf["pe_ltp"]

ocdf["ce_ac_ex_diff"] = ocdf["ce_actual_chg_pc"] - ocdf["ec_ce_pc"]
ocdf["pe_ac_ex_diff"] = ocdf["pe_actual_chg_pc"] - ocdf["ec_pe_pc"]

KeyboardInterrupt: 

In [9]:
ocdf.to_pickle(pickle_file_name)

In [10]:
ocdf = pd.read_pickle(pickle_file_name)

In [15]:
print(f"before filtering size: {ocdf.shape[0]}")
MINIMUM_MARKET_UP = 40
market_up = ocdf.loc[ocdf.market_open_pt > MINIMUM_MARKET_UP]
print(f"market up size: {market_up.shape[0]}")

# positive_first_candle = market_up.loc[market_up.first_candle_change_pt > 0]
# print(f"positive first candle size: {positive_first_candle.shape[0]}")

# negative_first_candle = market_up.loc[market_up.first_candle_change_pt < 0]
# print(f"negative first candle size: {negative_first_candle.shape[0]}")

MIN_PREMIUM = 90
min_premium_df = market_up.loc[market_up.ce_ltp >= MIN_PREMIUM]
print(f"min_premium_df size: {min_premium_df.shape[0]}")

# Actual - expected diff is positive
ac_ex_diff_pos = min_premium_df.loc[min_premium_df.ce_ac_ex_diff < 0]
print(f"diff positive size: {ac_ex_diff_pos.shape[0]}")

# MIN_VOLUME_REQUIRED = 10000
MIN_VOLUME_REQUIRED = 300000000 # By value

# Minimum liquidity required
min_volume = ac_ex_diff_pos.loc[ac_ex_diff_pos.ce_volume > MIN_VOLUME_REQUIRED]
print(f"min_volume size: {min_volume.shape[0]}")

# # To study the nature, market's first candle is negative and premium still is green
# market_red = ac_ex_diff_pos.loc[ac_ex_diff_pos.first_candle_change_pt < 0]
# print(f"market_red size: {market_red.shape[0]}")

# more_than_ex = ocdf.loc[ocdf.ec_ce_pc < ocdf.ce_actual_chg_pc]
# print(f"more than ex size: {more_than_ex.shape[0]}")
# # positive_actual = ocdf.loc[(ocdf.ec_ce_pc < 0) & (ocdf.ce_actual_chg_pc > 0)]
# positive_actual = more_than_ex.loc[(more_than_ex.ce_actual_chg_pc > 0)]
# print(f"positive actual: {positive_actual.shape[0]}")
# # First premium candle is green
# first_green = positive_actual.loc[positive_actual.ce_first_candle_chg_pc > 0]
# print(f"first green size: {first_green.shape[0]}")

# Group by Strike Price and Trade date and order by ce_first_candle_chg_pc
# call_max_ch = pd.DataFrame(columns=["trade_date", "expiry", "strike_price", "ce_first_candle_chg_pc"])
call_max_ch = pd.DataFrame(columns=min_volume.columns)
call_grouped = min_volume.groupby(["trade_date", "expiry"])
for name, call_group in call_grouped:
    max_sorted = call_group.sort_values(by="ce_ac_ex_diff", ascending=True)
    # max_sorted = call_group.sort_values(by="ce_first_candle_chg_pc", ascending=False)
    max_row1 = max_sorted.iloc[0]
    call_max_ch = pd.concat([call_max_ch, max_row1.to_frame().T])
    if max_sorted.shape[0] > 1:
        max_row2 = max_sorted.iloc[1]
        call_max_ch = pd.concat([call_max_ch, max_row2.to_frame().T])
    if max_sorted.shape[0] > 2:
        max_row3 = max_sorted.iloc[2]
        call_max_ch = pd.concat([call_max_ch, max_row3.to_frame().T])

# call_max_ch = first_green.groupby(["trade_date", "expiry"])['ce_first_candle_chg_pc'].max()
print(f"size: {call_max_ch.shape[0]}")
# call_max_ch[["trade_date", "expiry", "market_open_pt", "first_candle_open", "first_candle_ac_ex_diff_pt", "ce_first_candle_chg_pc", "ce_first_candle_high_pc", "ce_ltp", "ce_first_candle_open", "ce_first_candle_high", "ce_first_candle_low", "ce_first_candle_close", "ce_oi", "ce_oi_chg"]]
# print(call_max_ch.loc[call_max_ch.ce_first_candle_chg_pc >= .02])
# success_pc = (call_max_ch.loc[call_max_ch.ce_first_candle_chg_pc >= .05]).shape[0] * 100 / call_max_ch.shape[0]
# call_max_ch.loc[:,"ce_first_candle_chg_pc"] = call_max_ch["ce_first_candle_chg_pc"].apply('{:.2%}'.format)
# call_max_ch.loc[:,"ce_first_candle_high_pc"] = call_max_ch["ce_first_candle_high_pc"].apply('{:.2%}'.format)
# print(success_pc)
call_max_ch[["trade_date", "expiry", "market_open_pt", "first_candle_open", "ce_ac_ex_diff", "ce_ltp"]]

before filtering size: 3444
market up size: 1624
min_premium_df size: 734
diff positive size: 629
min_volume size: 346
size: 168


,trade_date,expiry,market_open_pt,first_candle_open,ce_ac_ex_diff,ce_ltp
17800,2023-01-05 00:00:00,2023-01-05 00:00:00,65.1,18101.95,-0.097721,252.3
17900,2023-01-05 00:00:00,2023-01-05 00:00:00,65.1,18101.95,-0.002726,160.75
17550,2023-01-31 00:00:00,2023-02-02 00:00:00,64.2,17731.45,-0.059093,235.45
17450,2023-02-01 00:00:00,2023-02-02 00:00:00,129.05,17811.6,-0.242627,326.1
17600,2023-02-01 00:00:00,2023-02-02 00:00:00,129.05,17811.6,-0.114331,225.0
17400,2023-02-01 00:00:00,2023-02-02 00:00:00,129.05,17811.6,-0.046478,365.0
17650,2023-02-03 00:00:00,2023-02-09 00:00:00,110.0,17721.75,-0.181773,145.0
17550,2023-02-03 00:00:00,2023-02-09 00:00:00,110.0,17721.75,-0.156047,198.8
17500,2023-02-03 00:00:00,2023-02-09 00:00:00,110.0,17721.75,-0.130357,230.0
17950,2023-02-16 00:00:00,2023-02-16 00:00:00,71.55,18094.75,-0.365275,92.25


In [16]:
call_max_ch[["trade_date", "expiry", "market_open_pt", "first_candle_open", "ce_ac_ex_diff", "ce_ltp"]].to_csv("predicted_results.csv")

In [ ]:
oc_columns = [
# "strike_price",
"trade_date",
"expiry",
"oc_date",
"market_open",
"market_open_pt",
"market_open_pc",
"previous_trading_open",
"previous_trading_close",
"first_candle_change_pt",
"first_candle_change_pc",
"ec_ce_pt",
"ec_ce_pc",
"ce_actual_chg_pt",
"ce_actual_chg_pc",
"ce_ac_ex_diff",
"ec_pe_pt",
"ec_pe_pc",
"pe_ac_ex_diff",
"pe_actual_chg_pt",
"pe_actual_chg_pc",
"first_candle_open",
"first_candle_high",
"first_candle_low",
"first_candle_close",
"ce_first_candle_chg_pt",
"ce_first_candle_chg_pc",
"pe_first_candle_chg_pt",
"pe_first_candle_chg_pc",
"ce_first_candle_open",
"ce_first_candle_high",
"ce_first_candle_low",
"ce_first_candle_close",
"ce_first_candle_volume",
"pe_first_candle_open",
"pe_first_candle_high",
"pe_first_candle_low",
"pe_first_candle_close",
"pe_first_candle_volume",
"previous_trading_day",
"ce_build_up",
"ce_trend",
"ce_time",
"ce_vega",
"ce_theta",
"ce_gamma",
"ce_delta",
"ce_iv_chg_pc",
"ce_iv_chg",
"ce_iv",
"ce_oi_chg_pc",
"ce_oi_chg",
"ce_oi",
"ce_volume_chg_pc",
"ce_volume_chg",
"ce_volume",
"ce_int_val",
"ce_ext_val",
"ce_ohol",
"ce_ltp_chg_pc",
"ce_ltp_chg",
"ce_ltp",
"ce_vwap",
"ce_bid",
"ce_ask",
"pe_ce_oi",
"pe_ce_oi_chg",
"pe_bid",
"pe_ask",
"pe_vwap",
"pe_ltp",
"pe_ltp_chg",
"pe_ltp_chg_pc",
"pe_ohol",
"pe_int_val",
"pe_ext_val",
"pe_volume",
"pe_volume_chg",
"pe_volume_chg_pc",
"pe_oi",
"pe_oi_chg",
"pe_oi_chg_pc",
"pe_iv",
"pe_iv_chg",
"pe_iv_chg_pc",
"pe_delta",
"pe_gamma",
"pe_theta",
"pe_vega",
"pcr_oi",
"pcr_oi_chg",
"pcr_vol",
"pe_time",
"pe_trend",
"pe_build_up",
]
ocdf.to_csv("test_final_output.csv", columns=oc_columns)